In [431]:
import pandas as pd
import os,re,sys
import requests
import logging
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
import string, re,os
import pickle,logging
import requests,json
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem.porter import PorterStemmer
from sklearn.utils import resample



In [432]:
df=pd.read_csv("sample30.csv")
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [433]:
def preprocess(document):
    #lowercase
    document = str(document).lower()
    stop_words = stopwords.words("english")
    stemmer = PorterStemmer()
    words = word_tokenize(document)
    words = [stemmer.stem(word) for word in words if word not in stop_words and word not in string.punctuation and re.sub("\d.*", "", word)]
    document = " ".join(words)
    return document


In [434]:
reviews=df[['name','reviews_username','reviews_rating','reviews_text','reviews_title','user_sentiment']]

In [436]:
reviews=reviews[~reviews['reviews_username'].isna()]
reviews=reviews[~reviews['reviews_title'].isna()]
reviews=reviews[~reviews['user_sentiment'].isna()]
reviews.isna().sum()


name                0
reviews_username    0
reviews_rating      0
reviews_text        0
reviews_title       0
user_sentiment      0
dtype: int64

### Preprocess the review title and review text

In [437]:
reviews['reviews_text1']=reviews['reviews_text'].apply(preprocess)
reviews['reviews_title1']=reviews['reviews_title'].apply(preprocess)
reviews.head()


In [438]:
reviews.head()

,name,reviews_username,reviews_rating,reviews_text,reviews_title,user_sentiment,reviews_text1,reviews_title1
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),joshua,5,i love this album. it's very good. more to the...,Just Awesome,Positive,love album 's good hip hop side current pop so...,awesom
1,Lundberg Organic Cinnamon Toast Rice Cakes,dorothy w,5,Good flavor. This review was collected as part...,Good,Positive,good flavor review collect part promot,good
2,Lundberg Organic Cinnamon Toast Rice Cakes,dorothy w,5,Good flavor.,Good,Positive,good flavor,good
3,K-Y Love Sensuality Pleasure Gel,rebecca,1,I read through the reviews on here before look...,Disappointed,Negative,read review look buy one coupl lubric ultim di...,disappoint
4,K-Y Love Sensuality Pleasure Gel,walker557,1,My husband bought this gel for us. The gel cau...,Irritation,Negative,husband bought gel us gel caus irrit felt like...,irrit


### Address the data imbalance with respect to user_sentiment

In [439]:
def address_imbalance_binary(reviews):
    col='user_sentiment'
    low = reviews[reviews[col] == 'Negative']
    high = reviews[reviews[col] == 'Positive']

    resample_low = resample(low, replace=True, n_samples=len(high), random_state=1)
    output = pd.concat([high, resample_low])
    output=output.sample(frac = 1)
    return output

In [441]:
# Call the data imbalance function to address the issue

print("-------------Before addressing the data imbalance---------------")
print(reviews.user_sentiment.value_counts())
reviews=address_imbalance_binary(reviews)
print("After addressing the data imbalance")
print(reviews.user_sentiment.value_counts())

(52824, 8)
Negative    26412
Positive    26412
Name: user_sentiment, dtype: int64


### Function to create classification model and save the pickle files for later use

In [443]:
def build_predict_save_model():
    reviews.head()
    X_train, X_test, y_train, y_test = train_test_split(reviews[['reviews_text1','reviews_title1']], reviews['user_sentiment'], random_state=1)
    vect=TfidfVectorizer()
    vect1=TfidfVectorizer()
    
    vect.fit(X_train['reviews_text1'])
    X_train_transformed=vect.fit_transform(X_train['reviews_text1'])
    X_test_transformed=vect.transform(X_test['reviews_text1'])
    
    vect1.fit(X_train['reviews_title1'])
    X_train_cats=vect1.fit_transform(X_train['reviews_title1'])
    X_test_cats=vect1.transform(X_test['reviews_title1'])
    
    x_train_final=hstack((X_train_transformed,X_train_cats))
    x_test_final=hstack((X_test_transformed,X_test_cats))

    rfc = RandomForestClassifier()
    rfc.fit(x_train_final, y_train)
    y_pred_class = rfc.predict(x_test_final)
    accuracy=str(round(metrics.accuracy_score(y_test, y_pred_class) * 100, 2)) + "%"
    print(f"category Model accuracy rfc: {accuracy}")

    
    with open("category_model_allG_rfc.pkl","wb") as f:
        pickle.dump(rfc, f) 

    with open("category_model_allG_vect.pkl", "wb") as f:
        pickle.dump(vect, f) 

    with open("category_model_allG_vect1.pkl", "wb") as f:
        pickle.dump(vect1, f)
        
    mnb = RandomForestClassifier()
    rfc.fit(x_train_final, y_train)
    y_pred_class = rfc.predict(x_test_final)
    accuracy=str(round(metrics.accuracy_score(y_test, y_pred_class) * 100, 2)) + "%"
    print(f"category Model accuracy rfc: {accuracy}")

    
    with open("category_model_allG_mnb.pkl","wb") as f:
        pickle.dump(rfc, f) 

    with open("category_model_allG_vect.pkl", "wb") as f:
        pickle.dump(vect, f) 

    with open("category_model_allG_vect1.pkl", "wb") as f:
        pickle.dump(vect1, f)

In [444]:
build_predict_save_model()

category Model accuracy rfc: 99.17%
category Model accuracy rfc: 99.13%


In [445]:
rfc=pickle.load(open("category_model_allG_rfc.pkl","rb"))
vect=pickle.load(open("category_model_allG_vect.pkl", "rb"))
vect1=pickle.load(open("category_model_allG_vect1.pkl", "rb"))
vect1

TfidfVectorizer()

In [446]:
title=vect.transform([preprocess("Disappointed")])
body=vect1.transform([preprocess("Just not Awesome")])

print(title)
print(body)
final=hstack((title,body))
category = rfc.predict(hstack((vect.transform([preprocess("Disappointed")]),vect1.transform([preprocess("Just not Awesome")]))))[0]
category

  (0, 2715)	1.0
  (0, 208)	1.0


'Negative'

In [462]:
test=reviews.sample(frac=1).head(20)

In [466]:
for line in test.itertuples():
    review_text=line[5]
    review_body=line[4]
    if rfc.predict(hstack((vect.transform([preprocess(review_body)]),vect1.transform([preprocess(review_text)]))))[0] == "Positive":
        print(line[1],line[6])

Sizzix Framelits Dies 1by Tim Holtz Bird Crazy-Silver Asst Sizes Positive
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total Positive
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total Positive
Pendaflex174 Divide It Up File Folder, Multi Section, Letter, Assorted, 12/pack Positive
Mrs. Meyer's174 Lemon Verbena Laundry Scent Booster - 18oz Positive
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital) Positive
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd Positive
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total Positive
Ragu Traditional Pasta Sauce Positive
Jason Aldean - They Don't Know Positive
Storkcraft Tuscany Glider and Ottoman, Beige Cushions, Espresso Finish Positive


In [458]:
test.head()

,name,reviews_username,reviews_rating,reviews_text,reviews_title,user_sentiment,reviews_text1,reviews_title1
15813,"Burt's Bees Lip Shimmer, Raisin",momo47,4,I am so disappointed that u dropped the cocoa ...,It's ok but it's not cocoa,Negative,disappoint u drop cocoa pleas pleas bring back...,'s ok 's cocoa
2079,Mike Dave Need Wedding Dates (dvd + Digital),billw,4,This is a amusing adult comedy with a crude un...,Amusing adult comedy,Negative,amus adult comedi crude underton,amus adult comedi
24614,Godzilla 3d Includes Digital Copy Ultraviolet ...,goku,5,This was much better then the 1998 one. Godzil...,This reboot didn't disappoint,Positive,much better one godzilla actual look like godz...,reboot n't disappoint
4622,The Resident Evil Collection 5 Discs (blu-Ray),danimal,4,if your looking to just buy all the movies at ...,great r.e. pack!,Negative,look buy movi,great r.e pack
13965,Clorox Disinfecting Wipes Value Pack Scented 1...,moep,5,"I have a container of wipes in every bathroom,...",Great product!,Negative,contain wipe everi bathroom kitchen garag even...,great product
